# Imports

In [1]:
import sys
sys.path.append('..')

In [2]:
from openmm import LangevinMiddleIntegrator
from openmm.app import AmberInpcrdFile, AmberPrmtopFile, PDBReporter, Simulation
from openmm.unit import kelvin, picoseconds
from openmm_numpy_reporters import ForceReporter, PositionReporter, PotentialEnergyReporter

# Simulation Parameters

In [3]:
TEMPERATURE = 300.0 * kelvin
FRICTION = 1 / picoseconds
INTEGRATION_TIME_STEP = 0.002 * picoseconds
SAVE_INTERVAL = 100
SIMULATION_STEPS = 1000 # 2ps simulation

# Running the Simulation

In [4]:
prmtop = AmberPrmtopFile('../data/adp/adp.prmtop')

system = prmtop.createSystem(
    implicitSolvent=None,
    constraints=None,
    nonbondedCutoff=None,
    hydrogenMass=None
)

inpcrd = AmberInpcrdFile('../data/adp/adp.crd')

integrator = LangevinMiddleIntegrator(TEMPERATURE, FRICTION, INTEGRATION_TIME_STEP)

In [5]:
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.getPositions(asNumpy=True))
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(TEMPERATURE)


Add a PDB reporter and the custom, appendable reporters to the Simulation.

In [6]:
simulation.reporters.append(PDBReporter('adp_simulation.pdb', SAVE_INTERVAL))
simulation.reporters.append(PositionReporter('positions.npy', SAVE_INTERVAL))
simulation.reporters.append(ForceReporter('forces.npy', SAVE_INTERVAL))
simulation.reporters.append(PotentialEnergyReporter('potential_energies.npy', SAVE_INTERVAL))

Run the simulation.

In [7]:
simulation.step(SIMULATION_STEPS)